In [14]:
# This script is used to make the "WHAM meta files" from the umbrella files.
# The Eric Theide python implementation of EMUS seems to work best with this format

import sys, os, os.path
import glob

import scipy as sp
import numpy as np

from emus import usutils as uu
from emus import emus, avar


import matplotlib
import matplotlib.pyplot as pp
from mpl_toolkits.mplot3d import Axes3D

import yt
from yt.frontends.boxlib.data_structures import AMReXDataset

from tempfile import TemporaryFile


%pylab inline


Populating the interactive namespace from numpy and matplotlib


In [15]:
# After phi_0 and \kappa data has been extracted using "Extract_Data.ipynb", load data from the folder 
# with umbrella files that have had this data removed. 



# the block below computes the number of samples in the umbrella files ( assuming they all have the same amount)
location='./umb_files' #data file location. A copy of the data should be used. 
list = sorted(os.listdir("./umb_files"))
Number_of_Umbrellas=len(list) 
dat = np.loadtxt('./umb_files/umbrella00000000.txt',usecols=[0],unpack=True) #any of the files should work
Number_of_Samples_in_Umb=dat.shape[0]


#load umbrella parameters
centers = np.loadtxt('phi0_centers.txt')
fks = np.loadtxt('spring_constants.txt')

# Additonal EMUS parameters should be set here
period=None
dim=1
T=0.01
k_B=1

In [16]:
#hold  umbrella data in 2D array with column index corresponding to umbrella index.
dat_array=numpy.zeros(shape=(Number_of_Samples_in_Umb,Number_of_Umbrellas),dtype=float64)
H_data=numpy.zeros(shape=(Number_of_Samples_in_Umb,Number_of_Umbrellas),dtype=float64)
No_umb_H_data=numpy.zeros(shape=(Number_of_Samples_in_Umb,Number_of_Umbrellas),dtype=float64)
i=0
for filename in list:    
    a=os.path.join(location, filename)
    Temp=np.loadtxt(a)
    dat_array[:,i]=Temp[:,0]
    H_data[:,i]=Temp[:,2]
    No_umb_H_data[:,i]=Temp[:,1]
    i=i+1

# Data is then reformatted that it is made compatible with EMUS struvture 
# This follows the format of their example in their github repo
# Essentially, every first index should correspond to all the data in that umbrella
cv_data=numpy.zeros(shape=(Number_of_Umbrellas,Number_of_Samples_in_Umb,),dtype=float64)
Hamiltonian=numpy.zeros(shape=(Number_of_Umbrellas,Number_of_Samples_in_Umb,),dtype=float64)
Ham_no_umb=numpy.zeros(shape=(Number_of_Umbrellas,Number_of_Samples_in_Umb,),dtype=float64)

for i in range (0,Number_of_Umbrellas):
    cv_data[i]=dat_array[0:Number_of_Samples_in_Umb,i]
    Hamiltonian[i]=H_data[0:Number_of_Samples_in_Umb,i]
    Ham_no_umb[i]=No_umb_H_data[0:Number_of_Samples_in_Umb,i]
    
    
list2=[0]*(len(list))
i=0
for filename in list:    
    list2[i]=os.path.join(location, filename)   
    i=i+1    
cv_traj=(cv_data)
list2=np.asarray(list2)

In [17]:
# A folder in the "WHAM meta file" format will be made
# This is a format that is standard for chemistry people
# EMUS is easier to work with in this format
# See WHAM documentation of grossfield for a precise description


# Switch location to a new copy of original data file with title denoting the WHAM format
location='./umb_files_Wham_Format' #data file location
list = sorted(os.listdir("./umb_files_Wham_Format"))
list3=[0]*(len(list))
i=0
for filename in list:    
    list3[i]=os.path.join(location, filename)   
    i=i+1

    
## code below is for removing the last two THREE of the data files(none phi average part).
## ONLY RUN ONCE on the NON-original data 

# temp=np.zeros(Number_of_Samples_in_Umb)    
# for filename in list3:    
#     with open(filename, 'r') as fin:
#         data = fin.read().splitlines(True)
#         for j in range (0,Number_of_Samples_in_Umb):
#             temp[j]=float(data[j][0:20])
#         with open(filename, 'w') as fout:
#             np.savetxt(fout, temp, fmt="%10.14f",delimiter='\t')



In [18]:
## Add time column to trimmed data set. 
##*****RUN ONCE******#

# DT=2.4414e-5 # time step used in run
# DT_vec=np.arange(Number_of_Samples_in_Umb)+1
# DT_vec=DT_vec*DT # create a vector of these timesteps (i.e dt,2*dt,3*dt,...)


# # Add time step column to data as a column before the phi averages column


# for filename in list3: 
#     with open(filename, 'r') as fin:
#         data = fin.read().splitlines(True)
#         data=np.asarray(data)
#         Place_holder = np.zeros(data.size, dtype=[('var1', float64), ('var2', float64)])
#         Place_holder['var1']=DT_vec
#         Place_holder['var2']=data
#         np.savetxt(filename, Place_holder, fmt="%10.12f %10.12f",delimiter='\t')

In [19]:
# Now we find the "middle" of the data where our loop is made.
# The location prints to screen

# Depending on the where the "turning point" of the average data is, find the max or min 
A=np.where(centers == centers.min())  # find min since we go from 1.0 to 0.74 to 1.0
# np.where(centers == centers.min())
A[0]

array([171])

In [20]:
# SET middle location index manually here
middle=171

In [21]:
# Overwrite copy of umbrella file with time step and phi average with WHAM format of data

#In this case we consider data with 2 pars. The 1.0 to 0.74 trip, and the 0.74 to 1.0 trip.

# We make two meta files for this

#part 1 (1.0 to 0.74)

list4=np.asarray(list3[0:middle+1])        
MAKE_WHAM_META_ARR = np.zeros(list4.size, dtype=[('var1', 'U60'), ('var2', float64), ('var3', float64)])
MAKE_WHAM_META_ARR['var1']=list4[0:middle+1]
MAKE_WHAM_META_ARR['var2']=centers[0:middle+1]
MAKE_WHAM_META_ARR['var3']=fks[0:middle+1]
np.savetxt('ONE_TO_074_META.txt', MAKE_WHAM_META_ARR, fmt="%60s %10.8f %10.8f",delimiter='\t')






In [22]:
#part 2 (0.74 to 1.0)

list5=np.asarray(list3[middle+1:])        
MAKE_WHAM_META_ARR_2 = np.zeros(list5.size, dtype=[('var1', 'U60'), ('var2', float64), ('var3', float64)])
MAKE_WHAM_META_ARR_2['var1']=list5
MAKE_WHAM_META_ARR_2['var2']=centers[middle+1:]
MAKE_WHAM_META_ARR_2['var3']=fks[middle+1:]
np.savetxt('074_TO_ONE_META.txt', MAKE_WHAM_META_ARR_2, fmt="%60s %10.8f %10.8f",delimiter='\t')



In [23]:
meta_file = 'ONE_TO_074_META.txt'         # Path to Meta File
psis, cv_trajs, neighbors = uu.data_from_meta(
    meta_file, dim, T=T, k_B=k_B, period=period)


meta_file = '074_TO_ONE_META.txt'         # Path to Meta File
psis, cv_trajs, neighbors = uu.data_from_meta(
    meta_file, dim, T=T, k_B=k_B, period=period)
